In [1]:
import numpy as np
words = set()
vectors = {}
with open('glove.6B.50d.txt', 'r', encoding="utf8", errors='ignore') as f:
    for line in f:
        line = line.strip().split()
        cword = line[0]
    
        try:
            v= np.array(line[1:], dtype=np.float64)
            if v.shape[0] == 50:
                vectors[cword]=v
                words.add(cword)
            else:
                print("Error in line: {}".format(line))
        except:
            print("Error in line: {}".format(line))
print("Number of read words: {}".format(len(words)))

Number of read words: 400000


In [2]:
vectors['zsombor']

array([-0.75898 , -0.47426 ,  0.4737  ,  0.7725  , -0.78064 ,  0.23233 ,
        0.046114,  0.84014 ,  0.24371 ,  0.022978,  0.53964 , -0.36101 ,
        0.94198 , -0.08498 , -0.17095 ,  0.52358 ,  0.38773 , -0.90057 ,
       -0.052854,  0.64844 , -0.28436 , -0.58268 ,  0.47154 , -0.15341 ,
        0.56085 ,  1.7462  ,  1.5916  , -0.60234 ,  0.60964 ,  0.83124 ,
       -1.1914  ,  0.093531,  0.33981 ,  0.23403 , -0.61819 , -0.68542 ,
        0.25952 , -0.27869 ,  0.035413,  0.58834 ,  0.45439 , -0.84254 ,
        0.1065  , -0.059397,  0.090449,  0.30581 , -0.61424 ,  0.78954 ,
       -0.014116,  0.6448  ])

In [3]:
def find_similar_word(word):
    word = word.lower()
    v1 = vectors[word]
    top = 5
    max_sim = np.ones(top)*-200
    mwords = np.chararray(top, itemsize=20, unicode=True)
    mwords[:] = None

    # cos hasonlosag
    u = np.sqrt(np.sum(np.square(v1)))

    for w in words:
        if w in word: 
            continue
        vm = vectors[w]
        uv = np.dot(v1,vm)
        v  = np.sqrt(np.sum(np.square(vm)))

        cos_sim = uv/(u*v)

        for i in range(top):
            if cos_sim > max_sim[i]:
                for j in range(top-1,i,-1):
                    max_sim[j] = max_sim[j-1]
                    mwords[j]  = mwords [j-1]
                mwords[i] = w
                max_sim[i] = cos_sim
                break

    return mwords,max_sim

In [4]:
mwords,max=find_similar_word("dollar")
print(mwords)
print(max)

['euro' 'currency' 'dollars' 'greenback' 'price']
[0.82643291 0.81990801 0.78832095 0.7816785  0.78095334]


In [5]:
def find_analogy(word1, word2, word3):
    word1=word1.lower()
    word2=word2.lower()
    word3=word3.lower()
    
    v1=vectors[word1]
    v2=vectors[word2]
    v3=vectors[word3]
    
    top = 5 
    max= np.ones(top)* -200
    mwords= np.chararray(top,itemsize=20,unicode=True)
    mwords[:]= None
    
    #cos similarity
    v21= v2 - v1
    
    u = np.sqrt(np.sum(np.square(v21)))
    
    for w in words:        
        if w in [word1, word2, word3] :
            continue
       
        vm=vectors[w] -v3
        uv = np.dot(v21,vm)
        v = np.sqrt(np.sum(np.square(vm)))
        cosine_sim=  uv/(u*v)
    
        
        for i in range(top):
            if cosine_sim > max[i]:
                for j in range(top-1,i,-1):
                    max[j]=max[j-1]
                    mwords[j]=mwords[j-1]
                #print(w)
                mwords[i]=w
                max[i]=cosine_sim
                break
            
        
    return mwords,max

In [6]:
def find_and_print_analogy(w1,w2,w3):
    print("Find analogy: {} - {} + {} =".format(w1,w2,w3))
    top5_words,values=find_analogy(w1,w2,w3)

    for i, w in enumerate(top5_words):
        print("{}. {:6.4f} : {}".format(i,values[i],w))

In [7]:
find_and_print_analogy("Hungary","Budapest","France")

Find analogy: Hungary - Budapest + France =
0. 0.8359 : paris
1. 0.6663 : ecole
2. 0.6548 : polytechnique
3. 0.6409 : école
4. 0.6311 : beaux-arts


In [8]:
find_and_print_analogy('king','man','queen')

Find analogy: king - man + queen =
0. 0.8397 : woman
1. 0.7796 : girl
2. 0.7039 : blonde
3. 0.6951 : boy
4. 0.6907 : blond
